In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_ch = pd.read_json("../data/parsed_ch.json")

In [ ]:
df_ch.head()

In [ ]:
df_ch = df_ch.transpose()
df_ch.head()

In [ ]:
df_0 = pd.read_json("../data/parsed_com0.json")
df_0 = df_0.transpose()
df_0.head()

In [ ]:
df_1 = pd.read_json("../data/parsed_com1.json")
df_1 = df_1.transpose()
df_2 = pd.read_json("../data/parsed_com2.json")
df_2 = df_2.transpose()
df_3 = pd.read_json("../data/parsed_com3.json")
df_3 = df_3.transpose()

In [ ]:
total_df = df_ch.append(other=[df_0])

In [ ]:
total_df.shape

## Clean-up

In [ ]:
links = total_df.index

structure of each line : `['https:', '', 'www.lafourchette.com', 'restaurant', 'joe-s', '218659']`

In [ ]:
restaurant_ids = []
restaurant_names = []
websites = []

for link in links.values:
    link_split = link.split("/")
    websites.append(link_split[2])
    restaurant_names.append(link_split[-2])
    restaurant_ids.append(link_split[-1])

In [ ]:
len(np.unique(restaurant_ids)) == len(restaurant_ids)

In [ ]:
total_df = total_df.set_index([restaurant_ids])

In [ ]:
total_df["website"] = websites

In [ ]:
total_df.shape